In [40]:
import pandas as pd
import re
import unidecode
import glob
import os
from sklearn import preprocessing
import geopy.distance

In [54]:
de_para_sigla = {'Acre' : 'AC', 
                'Alagoas': 'AL',
                'Amapá': 'AP',
                'Amazonas': 'AM',
                'Bahia': 'BA',
                'Ceará': 'CE',
                'Distrito Federal': 'DF',
                'Espírito Santo': 'ES',
                'Goiás': 'GO',
                'Maranhão': 'MA',
                'Mato Grosso': 'MT',
                'Mato Grosso do Sul': 'MS',
                'Minas Gerais':'MG',
                'Pará': 'PA',
                'Paraíba': 'PB',
                'Paraná': 'PR',
                'Pernambuco': 'PE',
                'Piauí': 'PI',
                'Rio de Janeiro': 'RJ',
                'Rio Grande do Norte': 'RN',
                'Rio Grande do Sul': 'RS',
                'Rondônia': 'RO',
                'Roraima': 'RR',
                'Santa Catarina': 'SC',
                'São Paulo': 'SP',
                'Sergipe': 'SE',
                'Tocantins': 'TO'}


def get_cities():
    cities = pd.read_excel('data/2018 12 12 _ database challenge.xlsx')
    cities = cities.drop('Unnamed: 0', 1)
    cities['Cidades'] = cities['Cidades'].apply(lambda x: re.sub(r'\([^)]*\)', '', x))
    cities['Cidades'] = cities['Cidades'].apply(lambda x: x.strip())
    cities['Cidades'] = cities['Cidades'].apply(lambda x: unidecode.unidecode(x))
    cities['Cidades'] = cities['Cidades'].str.upper()
    
    
    cities = cities.rename(columns={'Cidades': 'city', 'Cód Cidades': 'cod_cidades',
                                    'UF': 'uf', 'Tem malha aerea?':'tem_malha', 'área em KM2': 'area_em_km2',
                                   'Já é atendido pela Loggi?': 'atendido_loggi', 'Distância de Cajamar (km)': 'distancia_cajamar',
                                   'População 2018': 'populacao'}) 
    
    cities['distancia_cajamar'] = cities['distancia_cajamar'].astype(int)
    
    cities['tem_malha'] = cities['tem_malha'].map({'sim': 1, 'não':0})
    
    
    return cities

def get_airports():
    ### source https://raw.githubusercontent.com/jpatokal/openflights/master/data/airports.dat
    columns = ['indice', 'airport', 'city', 'country', 'tree', 'codigo', 'latitude_airport', 'longitude_airport', 'colum1', 'colum2', 'colum3', 'colum4', 'colum5', 'colum6']
    airports = pd.read_csv('data/airports.dat.txt', sep=',', names=columns)
    airports['city'] = airports['city'].str.upper()
    airports = airports[airports['country'] == 'Brazil']
    
    airports = airports[['city', 'latitude_airport', 'longitude_airport', 'airport', 'codigo']]
    
    return airports

def get_airport_with_uf(airports, municipios):
    dict_duplicated_airports = {'Belém/Brigadeiro Protásio de Oliveira Airport': 'PA',
                                'Val de Cans/Júlio Cezar Ribeiro International Airport': 'PA',
                                'Atlas Brasil Cantanhede Airport': 'RR', 
                                'Cascavel Airport': 'PR',
                                'Campo Grande Airport':'MS',
                                'Cruzeiro do Sul Airport': 'AC',
                                'Lagoa Santa Airport': 'MG',
                                'Plácido de Castro Airport': 'AC',
                                'Tabatinga Airport': 'AM',
                                'Maestro Wilson Fonseca Airport': 'PA',
                                'Iguatu Airport': 'CE',
                                'Brigadeiro Lysias Rodrigues Airport': 'TO',
                                'Santa Maria Airport': 'RS',
                                'Toledo Airport': 'PR',
                                'Santa Terezinha Airport': 'SC',
                                'Valença Airport': 'BA',
                                'Redenção Airport': 'PA',
                                'Humaitá Airport': 'AM',
                                'São Carlos Airport': 'SP',
                                'Canarana Airport': 'MT'}      
    
    airports = airports.merge(municipios, on='city')
    airports_without_duplicated = airports[~airports[['airport', 'city']].duplicated(keep=False)]
    airports_duplicated = airports[airports[['airport', 'city']].duplicated(keep=False)]


    for key in dict_duplicated_airports.keys():
        temp = airports_duplicated[(airports_duplicated['airport'] == key) & (airports_duplicated['UF'] == dict_duplicated_airports[key])]

        airports_without_duplicated = pd.concat([airports_without_duplicated, temp])
        
    return airports_without_duplicated
    
    
def get_frotas():
    ### source http://www.denatran.gov.br/estatistica/635-frota-2018
    frotas = pd.read_excel('data/frota.xls', skiprows=3)
    frotas = frotas[['UF', 'MUNICIPIO', 'CAMINHAO', 'CAMINHONETE', 'CAMIONETA', 'MOTOCICLETA']]
    
    frotas = frotas.rename(columns={'MUNICIPIO':'city', 'CAMINHAO': 'caminhao',
                                    'CAMINHONETE': 'caminhonete', 'CAMIONETA': 'camioneta',
                                    'MOTOCICLETA': 'motocicleta', 'UF': 'uf'})

    return frotas

def get_municipios():
    ### source https://ww2.ibge.gov.br/home/estatistica/populacao/contagem2007/popmunic2007layoutTCU14112007.xls
    municipios = pd.read_excel('data/municipios.xls', skiprows=3)
    municipios = municipios.rename(columns={'Unnamed: 3':'city', 'U.F': 'UF'})[['city', 'UF']]
    municipios['city'] = municipios['city'].str.upper()
    municipios['city'] = municipios['city'].apply(lambda x: unidecode.unidecode(x))
    municipios['city'] = municipios['city'].apply(lambda x: x.replace('*', ''))
    municipios['city'] = municipios['city'].apply(lambda x: x.strip())
    
    return municipios

def get_decolagens():
    ### source http://www.anac.gov.br/assuntos/dados-e-estatisticas/mercado-de-transporte-aereo/anuario-do-transporte-aereo/dados-do-anuario-do-transporte-aereo
    decolagens = pd.read_excel('data/decolagens.xlsx')
    decolagens = decolagens[~decolagens['Aeroporto'].str.contains('Total')]
    decolagens['Aeroporto'] = decolagens['Aeroporto'].apply(lambda x : x.split("-", maxsplit=1)[0])
    decolagens = decolagens.rename(columns= {'Aeroporto': 'codigo', 'Quantidade de Decolagens': 'quantidade_decolagens'})
    
    return decolagens

def get_gasolina():
    ### source http://www.anp.gov.br/precos-e-defesa/234-precos/levantamento-de-precos/4606-infopreco-precos-praticados-pelos-postos-revendedores
    gasolina = pd.read_excel('data/infopreco-30-11-2018.xlsx', skiprows=5)
    gasolina_mean = gasolina.groupby(['MUNICÍPIO', 'UF', 'PRODUTO'], as_index=False)['VALOR VENDA'].mean()
    gasolina_mean_pivot = pd.pivot_table(gasolina_mean, values='VALOR VENDA', index=['MUNICÍPIO', 'UF'], columns=['PRODUTO']).reset_index()
    gasolina_mean_pivot = gasolina_mean_pivot.rename(columns={'MUNICÍPIO': 'city', 'UF': 'uf', 'Gasolina C Comum': 'preco_gasolina',
                                                              'Diesel S10': 'preco_diesel_s10'})
    gasolina_mean_pivot = gasolina_mean_pivot[['uf', 'city', 'preco_gasolina', 'preco_diesel_s10']]
    
    gasolina_mean_pivot_uf = gasolina_mean_pivot.groupby('uf', as_index=False).mean()
    
    return gasolina_mean_pivot, gasolina_mean_pivot_uf

def get_area_urbana():
    # source https://pt.wikipedia.org/wiki/Lista_de_munic%C3%ADpios_do_Brasil_por_%C3%A1rea_urbana
    area_urbana = pd.read_excel('data/urbana.xlsx')
    area_urbana = area_urbana[['Município', 'Unidade federativa', 'Área urbana (km²)']]
    area_urbana['Unidade federativa'] = area_urbana['Unidade federativa'].str.replace('Pará Pará', 'Pará')
    area_urbana['Unidade federativa'] = area_urbana['Unidade federativa'].str.replace('Bahia Bahia', 'Bahia')
    area_urbana['Unidade federativa'] = area_urbana['Unidade federativa'].map(de_para_sigla)
    
    area_urbana['Município'] = area_urbana['Município'].apply(lambda x: unidecode.unidecode(x))
    area_urbana['Município'] = area_urbana['Município'].str.upper()
    
    area_urbana = area_urbana.rename(columns={'Unidade federativa': 'uf', 'Município': 'city',
                                              'Área urbana (km²)': 'area_urbana'})
    
    return area_urbana

def get_geo_municipios():
    ### https://github.com/kelvins/Municipios-Brasileiros
    geo_municipios = pd.read_csv('data/municipios_geolocalizacao.csv')
    geo_municipios = geo_municipios.rename(columns={'nome_municipio': 'city'})
    
    geo_municipios['city'] = geo_municipios['city'].apply(lambda x: unidecode.unidecode(x))
    geo_municipios['city'] = geo_municipios['city'].str.upper()
    
    return geo_municipios[['city', 'uf', 'latitude', 'longitude']]

def get_pib_municipio():
    ### https://pt.wikipedia.org/wiki/Lista_de_munic%C3%ADpios_do_Brasil_por_PIB
    pib_municipio = pd.read_excel('data/pib_municipio.xlsx', header=None)
    pib_municipio = pib_municipio.rename(columns={0:'city', 1:'uf', 2:'pib'})
    pib_municipio['pib'] = pib_municipio['pib'].apply(lambda x: str(x).strip().replace(" ", ""))
    pib_municipio['pib'] = pib_municipio['pib'].astype(float)
    
    pib_municipio['uf'] = pib_municipio['uf'].str.replace('Pará Pará', 'Pará')
    pib_municipio['uf'] = pib_municipio['uf'].str.replace('Bahia Bahia', 'Bahia')
    pib_municipio['uf'] = pib_municipio['uf'].map(de_para_sigla)
    
    pib_municipio['city'] = pib_municipio['city'].apply(lambda x: unidecode.unidecode(x))
    pib_municipio['city'] = pib_municipio['city'].str.upper()
    
    return pib_municipio

def get_sinistros():
    ### http://www2.susep.gov.br/menuestatistica/RankRoubo/menu1.asp
    sinistros = pd.DataFrame()

    for file in glob.glob("data/sinistros/*.xlsx"):
        temp_dataframe = pd.read_excel(file)
        temp_dataframe['uf'] = os.path.basename(file).replace('.xlsx', '')

        sinistros = pd.concat([sinistros, temp_dataframe])
        
    sinistros['uf'] = sinistros['uf'].map(de_para_sigla)
    sinistros = sinistros[sinistros['Modelo'].str.contains('CAMINHOES')]
    
    ## calculating weighted mean
    sinistros['sum_veiculos'] = sinistros.groupby('uf')['Veículos Expostos'].transform(sum)
    sinistros['factor'] = sinistros['(*) Índice de Roubos/Furtos (%)'] * sinistros['Veículos Expostos']
    sinistros['factor_sum'] = sinistros.groupby('uf')['factor'].transform(sum)
    sinistros['weighted_mean'] = sinistros['factor_sum'] / sinistros['sum_veiculos']
    sinistros.sort_values('weighted_mean', ascending=False)

    sinistros_mean = sinistros.groupby('uf', as_index=False)['weighted_mean'].first()
    sinistros_mean.sort_values('weighted_mean', ascending=False)
    sinistros_mean = sinistros_mean.rename(columns={'weighted_mean': 'indice_roubo'})

    return sinistros_mean

def fill_na_by_mean(data, columns):
    for column in columns:
        data[column] = data.groupby(['uf'])[column].apply(lambda x: x.fillna(x.mean()))
        
    return data

def set_scale_columns(data, columns):
    for column in columns:
        data['scaled_' + column] = preprocessing.minmax_scale(data[column])
        
    return data

def drop_scaled_columns(data, columns):
    for column in columns:
        data = data.drop(['scaled_' + column], 1)
        
    return data

def calculate_distance_airport(table):
    table['distance_airport'] = table.apply(lambda x: geopy.distance.geodesic((x['latitude_airport'], x['longitude_airport']), (x['latitude'], x['longitude'])).km, 1)
    table.loc[table['tem_malha'] == 1, 'distance_airport'] = 0
    
    return table
    
def calculate_score(table):
    ### CRITERIO
    ## viabilidade_terrestre - 15
    ## viabilidade_aerea - 20
    ## custo_indireto - 10
    ## custo_direto - 20
    ## lucro - 35
    
    # viabilidade_terrestre = caminhao, caminhonete, camioneta, motocicleta
    # viabilidade_aerea = quantidade_decolagens
    # custo_indireto = indice_roubo, preco_gasolina, preco_diesel_s10
    # custo_direto = distancia_cajamar, distance_airport
    # lucro = pib, area_urbana, idh_renda
    
    table['viabilidade_terrestre'] = (1*table['scaled_caminhao'] + 1*table['scaled_camioneta'] + 1*table['scaled_caminhonete'] + 1*table['scaled_motocicleta'])/4
    table['viabilidade_aerea'] = 1*table['scaled_quantidade_decolagens']
    table['custo_indireto'] = (1*table['scaled_indice_roubo'] + 1*table['scaled_preco_gasolina'] + 1*table['scaled_preco_diesel_s10'])/3
    table['custo_direto'] = (2*table['scaled_distancia_cajamar'] + 1* table['scaled_distance_airport'])/3
    table['lucro'] = (1*table['scaled_pib'] + 1*table['scaled_area_urbana'] + 1* table['idh_renda'])/3
    
    table['score'] = 0.15 * table['viabilidade_terrestre'] + 0.15 * table['viabilidade_aerea'] + 0.15 - table['custo_indireto'] + 0.20 - table['custo_direto'] + 0.35 * table['lucro']
    
    return table

def get_idh_renda():
    # http://www.atlasbrasil.org.br/2013/pt/ranking
    idh = pd.read_csv('data/idh.csv', encoding='latin-1', sep=';', skiprows=1)
    idh['city'], idh['uf'] = idh['Nome'].str.split('(', 1).str
    idh['uf'] = idh['uf'].str.replace(')', '')
    idh['city'] = idh['city'].apply(lambda x: x.strip())
    idh['city'] = idh['city'].apply(lambda x: unidecode.unidecode(x))
    idh['city'] = idh['city'].str.upper()
    
    idh = idh.rename(columns={'IDHM Renda (2010)': 'idh_renda'})
    
    return idh[['city', 'uf', 'idh_renda']]
    
def main():
    # loading tables
    airports = get_airports()
    cities = get_cities() 
    municipios = get_municipios()
    frotas = get_frotas()
    decolagens = get_decolagens()
    gasolina, gasolina_uf = get_gasolina()
    area_urbana = get_area_urbana()
    pib_municipio = get_pib_municipio()
    geo_municipios = get_geo_municipios()
    sinistros = get_sinistros()
    idh_renda = get_idh_renda()

    airports_uf = get_airport_with_uf(airports, municipios)
    airports_uf = airports_uf.rename(columns={'UF':'uf'})
    airports_uf = airports_uf.merge(decolagens, on='codigo', how='left')
    airports_uf = airports_uf.sort_values('quantidade_decolagens', ascending=False)
    airports_uf_decolagens = airports_uf.groupby('uf', as_index=False)['quantidade_decolagens'].first()
    airports_distance = airports_uf_decolagens.merge(airports_uf[['uf', 'quantidade_decolagens', 'latitude_airport', 'longitude_airport']], on=['uf', 'quantidade_decolagens'])
    
    cities = cities.merge(geo_municipios, on=['city', 'uf'])
    cities = cities.merge(idh_renda, on=['city', 'uf'])
    cities = cities.merge(sinistros, on='uf')
    cities = cities.merge(airports_distance, on='uf')
    cities = cities.merge(frotas, on=['city', 'uf'], how='left')
    cities = cities.merge(gasolina, on=['city', 'uf'], how='left')
    cities = cities.merge(area_urbana, on=['city', 'uf'], how='left')
    cities = cities.merge(pib_municipio, on=['city', 'uf'], how='left')
    cities['preco_gasolina'] = cities['preco_gasolina'].where(cities['preco_gasolina'].notnull(), cities['uf'].map(gasolina_uf.set_index('uf')['preco_gasolina'])).fillna(cities['preco_gasolina'].mean())
    cities['preco_diesel_s10'] = cities['preco_diesel_s10'].where(cities['preco_diesel_s10'].notnull(), cities['uf'].map(gasolina_uf.set_index('uf')['preco_diesel_s10'])).fillna(cities['preco_diesel_s10'].mean())
    cities = cities[cities['atendido_loggi'] == 'não']   
    
    
    cities = calculate_distance_airport(cities)
    #result = cities.merge(airports_uf, on=['city', 'uf'], how='left')
    
    columns = ['caminhao', 'camioneta', 'caminhonete', 'motocicleta', 'area_urbana', 'pib']
    cities = fill_na_by_mean(cities, columns)
    
    columns = ['caminhao', 'camioneta', 'caminhonete', 'motocicleta', 'area_urbana',
               'pib', 'quantidade_decolagens', 'indice_roubo', 'preco_gasolina', 
               'preco_diesel_s10', 'distancia_cajamar', 'distance_airport', 'idh_renda']
    
    cities = set_scale_columns(cities, columns)
    cities = calculate_score(cities)
    cities = drop_scaled_columns(cities, columns)
    cities = cities.sort_values('score', ascending=False)
    
    
    cities.to_csv('data/output/cidades.csv')
    
    return cities

In [ ]:
result = main()

In [50]:
result = main()

In [52]:
result.to_csv('cidades.csv')

In [43]:
result[result['uf'] != 'SP']

,cod_cidades,city,uf,atendido_loggi,distancia_cajamar,populacao,area_em_km2,tem_malha,latitude,longitude,...,preco_diesel_s10,area_urbana,pib,distance_airport,viabilidade_terrestre,viabilidade_aerea,custo_indireto,custo_direto,lucro,score
521,4208203,ITAJAI,SC,não,605,212632,289.255,0,-26.91010,-48.6705,...,3.348500,20.900000,1.712893e+07,85.040089,0.194422,0.138494,0.126762,0.123928,0.368852,0.278346
486,5002704,CAMPO GRANDE,MS,não,993,874184,8096.051,1,-20.44860,-54.6295,...,3.591000,332.000000,2.390214e+07,0.000000,0.803681,0.053336,0.330611,0.140896,0.729211,0.262270
516,4209102,JOINVILLE,SC,não,522,577024,1125.700,1,-26.30450,-48.8487,...,3.490722,114.400000,2.457085e+07,0.000000,0.433755,0.138494,0.293953,0.073179,0.512936,0.248232
308,4113700,LONDRINA,PR,não,537,558461,1650.809,1,-23.30400,-51.1691,...,3.426857,117.800000,1.582090e+07,0.000000,0.488941,0.269811,0.310481,0.075336,0.463719,0.240298
517,4205407,FLORIANOPOLIS,SC,não,696,485865,433.317,1,-27.59450,-48.5477,...,3.499000,31.900000,1.732853e+07,0.000000,0.402130,0.138494,0.260660,0.098196,0.411888,0.216399
309,4115200,MARINGA,PR,não,650,408678,487.930,1,-23.42050,-51.9333,...,3.426857,54.300000,1.423303e+07,0.000000,0.421327,0.269811,0.310481,0.091582,0.395458,0.190018
310,4119905,PONTA GROSSA,PR,não,515,344350,2067.545,1,-25.09160,-50.1668,...,3.426857,35.700000,1.159820e+07,0.000000,0.272506,0.269811,0.310481,0.072173,0.344235,0.169176
527,4202909,BRUSQUE,SC,não,634,128872,283.445,0,-27.09770,-48.9107,...,3.390000,12.800000,5.394835e+06,72.669869,0.139593,0.138494,0.214634,0.122451,0.297959,0.158914
248,3118601,CONTAGEM,MG,não,570,658632,194.586,0,-19.93210,-44.0539,...,3.499000,99.900000,2.627513e+07,35.124298,0.485507,0.396231,0.401943,0.096112,0.491361,0.156181
518,4202404,BLUMENAU,SC,não,603,348494,519.837,0,-26.91550,-49.0709,...,3.537000,44.000000,1.676711e+07,98.125150,0.312876,0.138494,0.275491,0.129613,0.401637,0.153175
